<a href="https://colab.research.google.com/github/mamekin05108/signatecup2024summer/blob/main/%E5%89%8D%E5%87%A6%E7%90%86v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re

# warningsを非表示にする
warnings.filterwarnings("ignore")

In [3]:
train = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/data/train.csv")
test = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/data/test.csv")

In [4]:
data = pd.concat([train, test], sort=False)

In [5]:
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

In [6]:
data.head()

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,customer_info,ProdTaken
0,0,50歳,Self Enquiry,2,900秒,Large Business,male,1.0,4.0,Basic,3.0,5,1,4,Executive,253905.0,未婚 車未所持 子供なし,1.0
1,1,56歳,Company Invited,1,14分,Salaried,Male,1.0,4.0,Standard,3.0,2,1,4,Senior Manager,404475.0,"離婚済み,車あり,子供無し",0.0
2,2,NaN,Self Enquiry,1,10分,Large Business,Female,1.0,3.0,Basic,3.0,4,0,4,Executive,278145.0,"結婚済み、自動車未所有,子供なし",1.0
3,3,三十七歳,Self Enquiry,2,1080秒,Small Business,female,1.0,3.0,Standard,4.0,1,0,5,Senior Manager,326805.0,離婚済み、車所持、子供無し,0.0
4,4,48歳,Company Invited,3,1020秒,Small Business,female,1.0,3.0,Basic,4.0,4,0,4,Executive,258435.0,独身／車所持／無子,1.0


In [7]:
def summarize_dataframe(df):
    rows = []
    for col in df.columns:
        dtype = df[col].dtype
        na_count = df[col].isnull().sum()
        unique_count = len(df[col].unique())
        na_rate = na_count / len(df) if len(df) > 0 else None
        rows.append([col, dtype, na_count, unique_count, na_rate])

    summary_df = pd.DataFrame(rows, columns=["列名", "列の型", "NaNである行の数", "値の種類", "NaNの率"])
    return summary_df

In [8]:
summary_df = summarize_dataframe(data)
summary_df

,列名,列の型,NaNである行の数,値の種類,NaNの率
0,id,int64,0,6978,0.000000
1,Age,object,193,257,0.027658
2,TypeofContact,object,18,3,0.002580
3,CityTier,int64,0,3,0.000000
4,DurationOfPitch,object,252,67,0.036113
5,Occupation,object,0,3,0.000000
6,Gender,object,0,18,0.000000
7,NumberOfPersonVisiting,float64,0,4,0.000000
8,NumberOfFollowups,float64,57,13,0.008169
9,ProductPitched,object,0,97,0.000000


とりあえずオブジェクトについてどれだけユニークがあるか

In [9]:
for col in ["Age","DurationOfPitch","Gender","ProductPitched","NumberOfTrips","Designation","MonthlyIncome","customer_info","ProdTaken"]:
    print(col, train[col].unique())

Age ['50歳' '56歳' nan '三十七歳' '48歳' '19歳' '47歳' '49歳' '33歳' '３１歳' '51歳' '46歳'
 '４９歳' '53才' '20歳' '25歳' '五十三歳' '20代' '44歳' '30歳' '34歳' '29歳' '30代' '41歳'
 '50代' '52歳' '22歳' '３６歳' '28歳' '25才' '五十一歳' '48才' '38才' '三十三歳' '42際' '５０代'
 '35才' '59歳' '四十三歳' '32歳' '４２才' '53歳' '58歳' '40代' '二十六歳' '52際' '54歳' '59才'
 '29才' '21才' '57歳' '37歳' '35歳' '36才' '38歳' '３７歳' '50才' '51才' '43歳' '四十歳'
 '26歳' '31歳' '27歳' '40歳' '３４歳' '40際' '21歳' '24才' '45歳' '36歳' '41才' '38際'
 '18歳' '39歳' '４６歳' '４０代' '５５歳' '58際' '24歳' '41際' '三十一歳' '52才' '３０代' '28才'
 '55際' '60代' '27際' '３２歳' '60歳' '五十四歳' '29際' '三十四歳' '４７歳' '37際' '二十四歳'
 '四十七歳' '58才' '39才' '23才' '３３歳' '４２歳' '４５歳' '３９歳' '５４歳' '55歳' '42歳' '３８歳'
 '47才' '５１歳' '３９才' '51際' '二十七歳' '30才' '２６歳' '４０歳' '36際' '三十歳' '23歳' '二十二歳'
 '37才' '３０歳' '２８歳' '34才' '五十歳' '四十一歳' '５９歳' '３５歳' '２２歳' '３７才' '四十九歳' '42才'
 '４４歳' '２５歳' '44際' '33才' '28際' '２４才' '55才' '４８歳' '四十四歳' '31才' '２０代' '二十歳'
 '22際' '31際' '二十三歳' '39際' '59際' '３３才' '三十六歳' '10代' '２７歳' '49才' '４３歳' '32際'
 '４１歳' '33際' '二十五歳' '27才' '46際' '二十九歳

In [10]:
#!pip install mojimoji
#import mojimoji

### 年齢
- 漢数字
- nullあり

In [11]:
def clean_age_column(df):
    def fullwidth_to_halfwidth(s):
        """全角数字を半角数字に変換する"""
        return s.translate(str.maketrans('０１２３４５６７８９', '0123456789'))

    def extract_age(age_str):
        if pd.isna(age_str):
            return None

        kanji_to_number = {
            '一': 1, '二': 2, '三': 3, '四': 4, '五': 5,
            '六': 6, '七': 7, '八': 8, '九': 9, '十': 10,
            '百': 100
        }

        age_str = fullwidth_to_halfwidth(str(age_str))  # 全角を半角に変換

        result = 0
        temp_number = 0
        has_kanji = False

        for char in age_str:
            if char in kanji_to_number:
                num = kanji_to_number[char]
                if num == 10 or num == 100:
                    temp_number = max(temp_number, 1)
                    result += temp_number * num
                    temp_number = 0
                else:
                    temp_number = num
                has_kanji = True
            elif char.isdigit():
                if has_kanji:
                    result += temp_number
                    temp_number = 0
                result = result * 10 + int(char)
                has_kanji = False
            else:
                if temp_number > 0:
                    result += temp_number
                    temp_number = 0
                has_kanji = False

        if temp_number > 0:
            result += temp_number

        return result if result > 0 else None

    # 年齢を数値に変換して新しい列を作成
    df['Numeric_Age'] = df['Age'].apply(extract_age)

    # 各年代の最頻値を取得
    modes = {}
    for decade in range(10, 70, 10):
        mode_value = df[(df['Numeric_Age'] >= decade) & (df['Numeric_Age'] < decade + 10)]['Numeric_Age'].mode()
        modes[f'{decade}代'] = mode_value.iloc[0] if not mode_value.empty else None

    # 最頻値の確認
    for decade, mode_value in modes.items():
        print(f'{decade}の最頻値: {mode_value}')

    # 年齢層の最頻値で補完
    def get_mode_for_group(group_str):
        group_str = fullwidth_to_halfwidth(group_str)
        if '10代' in group_str:
            return modes['10代']
        elif '20代' in group_str:
            return modes['20代']
        elif '30代' in group_str:
            return modes['30代']
        elif '40代' in group_str:
            return modes['40代']
        elif '50代' in group_str:
            return modes['50代']
        elif '60代' in group_str:
            return modes['60代']
        else:
            return None

    df['Age'] = df['Age'].apply(lambda x: get_mode_for_group(x) if '代' in fullwidth_to_halfwidth(str(x)) else extract_age(x))
    df = df.drop(columns=['Numeric_Age'])

    return df

In [12]:
#def clean_data(df):
#    df = clean_age_column(df)
#    return df
#df_data = clean_data(data)

### DurationOfPitch
問題点
- 分と秒が混在していること →　全部「分」にできそう

In [13]:
def clean_DurationOfPitch_column(df):
    def convert_to_minutes(duration_str):
        if pd.isna(duration_str):
            return None

        duration_str = str(duration_str).strip()

        # 文字列の末尾が「分」の場合
        if duration_str.endswith('分'):
            try:
                minutes = float(duration_str.replace('分', ''))
                return int(minutes)
            except ValueError:
                return None

        # 文字列の末尾が「秒」の場合
        elif duration_str.endswith('秒'):
            try:
                seconds = float(duration_str.replace('秒', ''))
                return int(seconds // 60)  # 秒を分に変換
            except ValueError:
                return None

        # その他の場合はNoneを返す
        return None

    # データフレームのDurationOfPitch列をクリーンアップ
    df['DurationOfPitch'] = df['DurationOfPitch'].apply(convert_to_minutes)

    return df

In [14]:
#def clean_data(df):
#    df = clean_DurationOfPitch_column(df)
#    return df
#data = clean_data(data)

In [15]:
#data.head()

### Gender
- 表記にぶれ
 - 空白
 - 半角・全角混在
 - 大文字小文字


In [16]:
def clean_gender_column(df):
    def extract_gender(gender_str):
        if pd.isna(gender_str):
            return None

        # 全角文字を半角文字に変換するための変換テーブルを作成
        zenkaku = "０１２３４５６７８９ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚ　"
        hankaku = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz "
        zenkaku_to_hankaku = str.maketrans(zenkaku, hankaku)

        # 文字列を全角から半角に変換
        cleaned_str = str(gender_str).translate(zenkaku_to_hankaku)

        # 全ての空白文字（全角および半角）を一つの空白に置き換え、トリムして大文字に変換
        cleaned_str = re.sub(r'\s+', ' ', cleaned_str).strip().upper()

        cleaned_str = re.sub(r"[\u3000 \t]", "", cleaned_str)

        return cleaned_str

    df['Gender'] = df['Gender'].apply(extract_gender)
    return df


#train = clean_gender_column(train)
#test = clean_gender_column(test)

In [17]:
#for col in ["Gender_Cleaned"]:
#    print(col, test[col].unique())

### ProductPitched

In [18]:
def clean_ProductPitched_column(df):
    def extract_ProductPitched(ProductPitched_str):
        if pd.isna(ProductPitched_str):
            return None

        # 全角文字を半角文字に変換するための変換テーブルを作成
        zenkaku = "０１２３４５６７８９ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚ　"
        hankaku = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz "
        zenkaku_to_hankaku = str.maketrans(zenkaku, hankaku)

        # 文字列を全角から半角に変換
        cleaned_str = str(ProductPitched_str).translate(zenkaku_to_hankaku)

        # 全ての空白文字（全角および半角）を一つの空白に置き換え、トリムして大文字に変換
        cleaned_str = re.sub(r'\s+', ' ', cleaned_str).strip().upper()

        cleaned_str = re.sub(r"[\u3000 \t]", "", cleaned_str)

        # その他の文字
        henmae = "Α𝖺ВΒ𐊡Ꭰᗞ𝙳ΕꓢՏ𝘵Ѕ"
        henato = "AABBBDDDESSTS"
        zenkaku_to_hankaku = str.maketrans(henmae, henato)
        cleaned_str = str(cleaned_str).translate(zenkaku_to_hankaku)

        # 最初の2文字に基づいてプレフィックスを追加
        if cleaned_str.startswith('SU'):
            cleaned_str = 'SUPERDELUXE'
        elif cleaned_str.startswith('BA'):
            cleaned_str = 'BASIC'
        elif cleaned_str.startswith('ST'):
            cleaned_str = 'STANDARD'
        elif cleaned_str.startswith('DE'):
            cleaned_str = 'DELUXE'
        elif cleaned_str.startswith('KI'):
            cleaned_str = 'KING'
        return cleaned_str

    df['ProductPitched'] = df['ProductPitched'].apply(extract_ProductPitched)
    return df

#train = clean_ProductPitched_column(train)
#test = clean_ProductPitched_column(test)

In [19]:
#for col in ["ProductPitched_Cleaned"]:
#    print(col, test[col].unique())

### Designation

In [20]:
def clean_Designation_column(df):
    def extract_Designation(Designation_str):
        if pd.isna(Designation_str):
            return None

        # 全角文字を半角文字に変換するための変換テーブルを作成
        zenkaku = "０１２３４５６７８９ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚ　"
        hankaku = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz "
        zenkaku_to_hankaku = str.maketrans(zenkaku, hankaku)

        # 文字列を全角から半角に変換
        cleaned_str = str(Designation_str).translate(zenkaku_to_hankaku)

        # 文字列を全角から半角に変換
        cleaned_str = str(Designation_str).translate(zenkaku_to_hankaku)

        # 全ての空白文字（全角および半角）を一つの空白に置き換え、トリムして大文字に変換
        cleaned_str = re.sub(r'\s+', ' ', cleaned_str).strip().upper()

        cleaned_str = re.sub(r"[\u3000 \t]", "", cleaned_str)


        # その他の文字
        henmae = "АΑЕEＰ×ΜMSՏSЅ"
        henato = "AAEEPXMMSSSS"
        zenkaku_to_hankaku = str.maketrans(henmae, henato)
        cleaned_str = str(cleaned_str).translate(zenkaku_to_hankaku)


        # 最初の2文字に基づいてプレフィックスを追加
        if cleaned_str.startswith('AV'):
            cleaned_str = 'AVP'
        elif cleaned_str.startswith('SE'):
            cleaned_str = 'Senior Manager'
        elif cleaned_str.startswith('EX'):
            cleaned_str = 'Executive'
        elif cleaned_str.startswith('MA'):
            cleaned_str = 'Manager'
        elif cleaned_str.startswith('VP'):
            cleaned_str = 'VP'
        return cleaned_str

    df['Designation'] = df['Designation'].apply(extract_Designation)
    return df


### NumberOfTrips

In [21]:
def clean_NumberOfTrips_column(df):
    def extract_NumberOfTrips(NumberOfTrips_str):
        # 欠損値を処理
        if pd.isna(NumberOfTrips_str):
            return None

        # 日本語表現を数値にマッピング
        mapping = {
            '年に1回': 1,
            '年に2回': 2,
            '年に3回': 3,
            '年に4回': 4,
            '年に5回': 5,
            '年に6回': 6,
            '年に7回': 7,
            '年に8回': 8,
            '半年に1回': 2,  # 半年に1回は年に2回
            '四半期に1回': 4  # 四半期に1回は年に4回
        }

        # 日本語表現を数値に置換
        if NumberOfTrips_str in mapping:
            return mapping[NumberOfTrips_str]

        # 残りの文字列を数値に変換
        try:
            return float(NumberOfTrips_str)
        except ValueError:
            return None

    # データフレームのNumberOfTrips列をクリーンアップ
    df['NumberOfTrips'] = df['NumberOfTrips'].apply(extract_NumberOfTrips)

    # 列の型をfloat64に変換
    df['NumberOfTrips'] = df['NumberOfTrips'].astype('float64')
    return df


In [22]:
data.head()

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,customer_info,ProdTaken
0,0,50歳,Self Enquiry,2,900秒,Large Business,male,1.0,4.0,Basic,3.0,5,1,4,Executive,253905.0,未婚 車未所持 子供なし,1.0
1,1,56歳,Company Invited,1,14分,Salaried,Male,1.0,4.0,Standard,3.0,2,1,4,Senior Manager,404475.0,"離婚済み,車あり,子供無し",0.0
2,2,NaN,Self Enquiry,1,10分,Large Business,Female,1.0,3.0,Basic,3.0,4,0,4,Executive,278145.0,"結婚済み、自動車未所有,子供なし",1.0
3,3,三十七歳,Self Enquiry,2,1080秒,Small Business,female,1.0,3.0,Standard,4.0,1,0,5,Senior Manager,326805.0,離婚済み、車所持、子供無し,0.0
4,4,48歳,Company Invited,3,1020秒,Small Business,female,1.0,3.0,Basic,4.0,4,0,4,Executive,258435.0,独身／車所持／無子,1.0


### MonthlyIncome
- 月収丸々万円がいる

In [23]:
def clean_MonthlyIncome_column(df):
    def extract_MonthlyIncome(MonthlyIncome_str):
        if pd.isna(MonthlyIncome_str):
            return None

        # 文字列を小文字に変換（任意）
        #MonthlyIncome_str = str(MonthlyIncome_str).lower()

        # 数字と単位を抽出
        match = re.search(r'([\d.,]+)\s*(万|千)?', MonthlyIncome_str)

        if match:
            number_str, unit = match.groups()
            number_str = number_str.replace(',', '')  # カンマを削除
            number = float(number_str)

            # 単位に応じて数値を変換
            if unit == '万':
                income = number * 10000
            elif unit == '千':
                income = number * 1000
            else:
                income = number
        else:
            # マッチしない場合は NaN
            income = None

        return income

    df['MonthlyIncome'] = df['MonthlyIncome'].apply(extract_MonthlyIncome)
    return df

# trainがあなたのDataFrameであると仮定してい#ます
#train = clean_MonthlyIncome_column(train)
#test = clean_MonthlyIncome_column(test)

### customer_info
- 結婚
- 車
- 子供  

に分けることができそう。ただしいずれも表示にぶれあり  
区切りもスペース改行スラッシュコンマといろいろなのでいったん何かに置換すべき

In [24]:
def parse_customer_info(info):
    if pd.isna(info):
        return None, None, None

    # 各項目の初期値を設定
    marital_status = None
    car_ownership = None
    children = None

    # 婚姻状況の抽出
    if re.search(r'未婚', info):
        marital_status = '未婚'
    elif re.search(r'独身', info):
        marital_status = '独身'
    elif re.search(r'結婚|既婚', info):
        marital_status = '結婚'
    elif re.search(r'離婚', info):
        marital_status = '離婚'

    # 車の所有の抽出
    if re.search(r'車(未所持|未所有|保有なし|なし)', info):
        car_ownership = 'なし'
    elif re.search(r'車(あり|所持|保有|所有)', info):
        car_ownership = 'あり'

    # 子供の有無の抽出
    children = re.search(r'子供[無な|無し|ゼロ]|こども[無な|無し]|無子|非育児家庭', info)
    if children:
        children = 0
    else:
        children = re.search(r'子供有り|こども有り|子供[0-9]+人|こども[0-9]+人|[0-9]児', info)
        if children:
            children = re.search(r'[0-9]+', info)
            children = children.group(0) if children else 'あり'

    return marital_status, car_ownership, children

def clean_customer_info(df):
    df[['MaritalStatus', 'OwnCar', 'NumberOfChildren']] = df['customer_info'].apply(lambda x: pd.Series(parse_customer_info(x)))
    return df


### NumberOfFollowups
- 100以上の数字がある

In [25]:
def clean_NumberOfFollowups_column(df):
    df['NumberOfFollowups'] = df['NumberOfFollowups'].apply(lambda x: x / 100 if x >= 100 else x)
    return df

### 年齢層・収入層

def Agebin_column(df):
  df['Agebin'] = pd.cut(df['Age'], bins = [0,25, 31, 40, 50, 65], labels = ['0-25','26-30', '31-40', '41-50', '51-65'])
  return df

def Incomebin_column(df):
  df['Incomebin'] = pd.cut(df['MonthlyIncome'], bins = [0,200000,250000, 300000, 350000,400000,450000,500000,550000,600000], labels = ['<200000', '<250000', '<300000', '<350000','<400000','<450000','<500000','<550000','<600000'])
  return df

## でいったんきれいにした

In [26]:
def clean_data(df):
    df = clean_age_column(df)
    df = clean_DurationOfPitch_column(df)
    df = clean_gender_column(df)
    df = clean_ProductPitched_column(df)
    df = clean_Designation_column(df)
    df = clean_NumberOfTrips_column(df)
    df = clean_MonthlyIncome_column(df)
    df = clean_customer_info(df)
    df = clean_NumberOfFollowups_column(df)
#   df = Agebin_column(df)
#    df = Incomebin_column(df)
    df = df.drop(columns=['customer_info'])
    return df

data = clean_data(data)


10代の最頻値: 19.0
20代の最頻値: 28.0
30代の最頻値: 30.0
40代の最頻値: 40.0
50代の最頻値: 50.0
60代の最頻値: 60.0


In [27]:
summary_df = summarize_dataframe(data)
summary_df

,列名,列の型,NaNである行の数,値の種類,NaNの率
0,id,int64,0,6978,0.000000
1,Age,float64,193,45,0.027658
2,TypeofContact,object,18,3,0.002580
3,CityTier,int64,0,3,0.000000
4,DurationOfPitch,float64,252,34,0.036113
5,Occupation,object,0,3,0.000000
6,Gender,object,0,2,0.000000
7,NumberOfPersonVisiting,float64,0,4,0.000000
8,NumberOfFollowups,float64,57,7,0.008169
9,ProductPitched,object,0,5,0.000000


In [28]:
  # 列の型をfloat64に変換
  data['NumberOfChildren'] = data['NumberOfChildren'].astype('float64')

In [29]:
df_train = data[:len(train)]
df_test = data[len(train):]

In [30]:
nan_records = data[data["NumberOfChildren"].isna()]
nan_records

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,MaritalStatus,OwnCar,NumberOfChildren
136,136,46.0,Self Enquiry,2,9.0,Large Business,FEMALE,2.0,3.0,SUPERDELUXE,4.0,2.0,0,2,AVP,483855.0,0.0,独身,あり,NaN
153,153,50.0,Self Enquiry,1,36.0,Small Business,MALE,3.0,4.0,SUPERDELUXE,3.0,3.0,0,1,AVP,490000.0,0.0,結婚,あり,NaN
174,174,55.0,Self Enquiry,1,24.0,Small Business,MALE,2.0,3.0,SUPERDELUXE,3.0,7.0,0,2,AVP,428025.0,0.0,結婚,あり,NaN
243,243,45.0,Self Enquiry,1,7.0,Salaried,MALE,2.0,4.0,SUPERDELUXE,4.0,6.0,0,1,AVP,560000.0,0.0,独身,なし,NaN
350,350,30.0,Self Enquiry,1,10.0,Large Business,FEMALE,2.0,4.0,SUPERDELUXE,4.0,1.0,0,3,AVP,510480.0,0.0,離婚,あり,NaN
355,355,28.0,Self Enquiry,2,8.0,Salaried,MALE,3.0,4.0,KING,3.0,3.0,0,3,VP,495570.0,0.0,未婚,あり,NaN
374,374,47.0,Self Enquiry,1,8.0,Salaried,FEMALE,3.0,3.0,SUPERDELUXE,3.0,4.0,0,2,AVP,498870.0,0.0,離婚,あり,NaN
440,440,46.0,Self Enquiry,2,17.0,Salaried,FEMALE,3.0,5.0,SUPERDELUXE,3.0,5.0,0,3,Manager,442125.0,0.0,結婚,あり,NaN
457,457,40.0,Company Invited,1,15.0,Small Business,MALE,2.0,3.0,KING,3.0,5.0,0,2,Senior Manager,503670.0,0.0,離婚,あり,NaN
475,475,44.0,Self Enquiry,1,10.0,Large Business,MALE,4.0,5.0,KING,3.0,3.0,0,1,VP,464160.0,0.0,独身,あり,NaN


In [31]:
#df_test.to_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_test.csv", index=False)
#df_train.to_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_train.csv", index=False)

In [32]:
cat_cols = ['Designation','ProdTaken', 'OwnCar', 'Passport',
            'CityTier','MaritalStatus',
            'ProductPitched','Gender','Occupation','TypeofContact'
            ]

# 欠損MonthlyIncome

In [33]:
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [34]:
# カテゴリカル変数の列名を指定
categorical_features = ['TypeofContact', 'Occupation', 'Gender', 'ProductPitched', 'Designation', 'MaritalStatus', 'OwnCar']

# カテゴリカル変数をカテゴリ型に変換
for col in categorical_features:
    data[col] = data[col].astype('category')


In [35]:
# 欠損値がないサンプルを取得
train_data = data[data['MonthlyIncome'].notnull()]
X_train = train_data.drop(columns=['MonthlyIncome', 'ProdTaken'])
y_train = train_data['MonthlyIncome']

# 交差検証の準備
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# 評価指標の初期化
mse_scores = []
mae_scores = []
r2_scores = []

for train_index, val_index in kf.split(X_train):
    X_train_cv, X_val_cv = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_cv, y_val_cv = y_train.iloc[train_index], y_train.iloc[val_index]

    # LightGBMモデルの訓練
    model = lgb.LGBMRegressor(categorical_feature=categorical_features)
    model.fit(X_train_cv, y_train_cv, categorical_feature=categorical_features)

    # バリデーションデータで予測
    y_pred = model.predict(X_val_cv)

    # 評価指標の計算
    mse_scores.append(mean_squared_error(y_val_cv, y_pred))
    mae_scores.append(mean_absolute_error(y_val_cv, y_pred))
    r2_scores.append(r2_score(y_val_cv, y_pred))

# 平均評価指標の表示
print(f"Mean Squared Error: {sum(mse_scores)/len(mse_scores)}")
print(f"Mean Absolute Error: {sum(mae_scores)/len(mae_scores)}")
print(f"R² Score: {sum(r2_scores)/len(r2_scores)}")

# モデルの最終訓練
model = lgb.LGBMRegressor(categorical_feature=categorical_features)
model.fit(X_train, y_train, categorical_feature=categorical_features)

# 欠損値の予測
test_data = data[data['MonthlyIncome'].isnull()]
X_test = test_data.drop(columns=['MonthlyIncome', 'ProdTaken'])
predicted_income = model.predict(X_test)

# 欠損値を補完
data.loc[data['MonthlyIncome'].isnull(), 'MonthlyIncome'] = predicted_income


[LightGBM] [Warning] categorical_feature is set=TypeofContact,Occupation,Gender,ProductPitched,Designation,MaritalStatus,OwnCar, categorical_column=2,5,6,9,14,15,16 will be ignored. Current value: categorical_feature=TypeofContact,Occupation,Gender,ProductPitched,Designation,MaritalStatus,OwnCar
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000670 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 409
[LightGBM] [Info] Number of data points in the train set: 5498, number of used features: 18
[LightGBM] [Info] Start training from score 356531.383230
[LightGBM] [Warning] categorical_feature is set=TypeofContact,Occupation,Gender,ProductPitched,Designation,MaritalStatus,OwnCar, categorical_column=2,5,6,9,14,15,16 will be ignored. Current value: categorical_feature=TypeofContact,Occupation,Gender,ProductPitched,Designation,MaritalStatus,OwnC

In [36]:
# 欠損値がないサンプルを取得
train_data = data[data['Age'].notnull()]
X_train = train_data.drop(columns=['Age', 'ProdTaken'])
y_train = train_data['Age']

# 交差検証の準備
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# 評価指標の初期化
mse_scores = []
mae_scores = []
r2_scores = []

for train_index, val_index in kf.split(X_train):
    X_train_cv, X_val_cv = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_cv, y_val_cv = y_train.iloc[train_index], y_train.iloc[val_index]

    # LightGBMモデルの訓練
    model = lgb.LGBMRegressor(categorical_feature=categorical_features)
    model.fit(X_train_cv, y_train_cv, categorical_feature=categorical_features)

    # バリデーションデータで予測
    y_pred = model.predict(X_val_cv)

    # 評価指標の計算
    mse_scores.append(mean_squared_error(y_val_cv, y_pred))
    mae_scores.append(mean_absolute_error(y_val_cv, y_pred))
    r2_scores.append(r2_score(y_val_cv, y_pred))

# 平均評価指標の表示
print(f"Mean Squared Error: {sum(mse_scores)/len(mse_scores)}")
print(f"Mean Absolute Error: {sum(mae_scores)/len(mae_scores)}")
print(f"R² Score: {sum(r2_scores)/len(r2_scores)}")

# モデルの最終訓練
model = lgb.LGBMRegressor(categorical_feature=categorical_features)
model.fit(X_train, y_train, categorical_feature=categorical_features)

# 欠損値の予測
test_data = data[data['Age'].isnull()]
X_test = test_data.drop(columns=['Age', 'ProdTaken'])
predicted_income = model.predict(X_test)

# 欠損値を補完
data.loc[data['Age'].isnull(), 'Age'] = predicted_income

[LightGBM] [Warning] categorical_feature is set=TypeofContact,Occupation,Gender,ProductPitched,Designation,MaritalStatus,OwnCar, categorical_column=1,4,5,8,13,15,16 will be ignored. Current value: categorical_feature=TypeofContact,Occupation,Gender,ProductPitched,Designation,MaritalStatus,OwnCar
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000642 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 619
[LightGBM] [Info] Number of data points in the train set: 5428, number of used features: 18
[LightGBM] [Info] Start training from score 38.888909
[LightGBM] [Warning] categorical_feature is set=TypeofContact,Occupation,Gender,ProductPitched,Designation,MaritalStatus,OwnCar, categorical_column=1,4,5,8,13,15,16 will be ignored. Current value: categorical_feature=TypeofContact,Occupation,Gender,ProductPitched,Designation,MaritalStatus,OwnCar
[

In [37]:
# 欠損値がないサンプルを取得
train_data = data[data['DurationOfPitch'].notnull()]
X_train = train_data.drop(columns=['DurationOfPitch', 'ProdTaken'])
y_train = train_data['DurationOfPitch']

# 交差検証の準備
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# 評価指標の初期化
mse_scores = []
mae_scores = []
r2_scores = []

for train_index, val_index in kf.split(X_train):
    X_train_cv, X_val_cv = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_cv, y_val_cv = y_train.iloc[train_index], y_train.iloc[val_index]

    # LightGBMモデルの訓練
    model = lgb.LGBMRegressor(categorical_feature=categorical_features)
    model.fit(X_train_cv, y_train_cv, categorical_feature=categorical_features)

    # バリデーションデータで予測
    y_pred = model.predict(X_val_cv)

    # 評価指標の計算
    mse_scores.append(mean_squared_error(y_val_cv, y_pred))
    mae_scores.append(mean_absolute_error(y_val_cv, y_pred))
    r2_scores.append(r2_score(y_val_cv, y_pred))

# 平均評価指標の表示
print(f"Mean Squared Error: {sum(mse_scores)/len(mse_scores)}")
print(f"Mean Absolute Error: {sum(mae_scores)/len(mae_scores)}")
print(f"R² Score: {sum(r2_scores)/len(r2_scores)}")

# モデルの最終訓練
model = lgb.LGBMRegressor(categorical_feature=categorical_features)
model.fit(X_train, y_train, categorical_feature=categorical_features)

# 欠損値の予測
test_data = data[data['DurationOfPitch'].isnull()]
X_test = test_data.drop(columns=['DurationOfPitch', 'ProdTaken'])
predicted_income = model.predict(X_test)

# 欠損値を補完
data.loc[data['DurationOfPitch'].isnull(), 'DurationOfPitch'] = predicted_income

[LightGBM] [Warning] categorical_feature is set=TypeofContact,Occupation,Gender,ProductPitched,Designation,MaritalStatus,OwnCar, categorical_column=2,4,5,8,13,15,16 will be ignored. Current value: categorical_feature=TypeofContact,Occupation,Gender,ProductPitched,Designation,MaritalStatus,OwnCar
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000664 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 671
[LightGBM] [Info] Number of data points in the train set: 5380, number of used features: 18
[LightGBM] [Info] Start training from score 14.483643
[LightGBM] [Warning] categorical_feature is set=TypeofContact,Occupation,Gender,ProductPitched,Designation,MaritalStatus,OwnCar, categorical_column=2,4,5,8,13,15,16 will be ignored. Current value: categorical_feature=TypeofContact,Occupation,Gender,ProductPitched,Designation,MaritalStatus,OwnCar
[

In [38]:
summary_df = summarize_dataframe(data)
summary_df

,列名,列の型,NaNである行の数,値の種類,NaNの率
0,id,int64,0,6978,0.000000
1,Age,float64,0,237,0.000000
2,TypeofContact,category,18,3,0.002580
3,CityTier,int64,0,3,0.000000
4,DurationOfPitch,float64,0,285,0.000000
5,Occupation,category,0,3,0.000000
6,Gender,category,0,2,0.000000
7,NumberOfPersonVisiting,float64,0,4,0.000000
8,NumberOfFollowups,float64,57,7,0.008169
9,ProductPitched,category,0,5,0.000000


In [39]:
data.tail()

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,MaritalStatus,OwnCar,NumberOfChildren
3484,6973,41.0,Company Invited,1,14.0,Small Business,FEMALE,1.0,3.0,BASIC,3.0,2.0,0,4,Executive,261840.0,NaN,結婚,なし,0.0
3485,6974,44.0,Company Invited,1,35.0,Salaried,MALE,3.0,5.0,DELUXE,3.0,3.0,0,3,Manager,349770.0,NaN,結婚,あり,1.0
3486,6975,24.0,Self Enquiry,2,21.0,Small Business,MALE,2.0,3.0,BASIC,3.0,2.0,0,3,Executive,270000.0,NaN,独身,なし,0.0
3487,6976,25.0,Self Enquiry,1,9.0,Small Business,MALE,2.0,3.0,BASIC,3.0,2.0,0,3,Executive,272430.0,NaN,結婚,なし,1.0
3488,6977,44.0,Self Enquiry,1,8.0,Small Business,FEMALE,4.0,4.0,DELUXE,3.0,5.0,0,1,Manager,411795.0,NaN,未婚,なし,3.0


In [40]:
# カテゴリカル変数をカテゴリ型からオブジェクト型に変換する関数
def convert_category_to_object(data, column_name):
    data[column_name] = data[column_name].astype('object')

# 変換するカテゴリカル変数の列名を指定
columns_to_convert = ['TypeofContact', 'Occupation', 'Gender', 'ProductPitched', 'Designation', 'MaritalStatus', 'OwnCar']

# 各カテゴリカル変数を変換
for column in columns_to_convert:
    convert_category_to_object(data, column)

# ＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝案１すべて数字にに置き換え＝＝＝＝＝＝＝＝＝＝＝


In [41]:
from sklearn.preprocessing import OneHotEncoder

In [42]:
summary_df = summarize_dataframe(data)
summary_df


,列名,列の型,NaNである行の数,値の種類,NaNの率
0,id,int64,0,6978,0.000000
1,Age,float64,0,237,0.000000
2,TypeofContact,object,18,3,0.002580
3,CityTier,int64,0,3,0.000000
4,DurationOfPitch,float64,0,285,0.000000
5,Occupation,object,0,3,0.000000
6,Gender,object,0,2,0.000000
7,NumberOfPersonVisiting,float64,0,4,0.000000
8,NumberOfFollowups,float64,57,7,0.008169
9,ProductPitched,object,0,5,0.000000


In [43]:
#TypeofContact
def typeof_contact_to_dummy(contact_type):
    # `Self Enquiry` は 1、`Company Invited` は 0 を返す
    return 1 if contact_type == 'Self Enquiry' else 0

In [44]:
#Occupation
def occupation_to_dummy(occupation):
    # Occupation に応じて対応する値を返す
    if occupation == 'Large Business':
        return 2
    elif occupation == 'Small Business':
        return 1
    elif occupation == 'Salaried':
        return 0

In [45]:
def gender_to_dummy(gender):
    # 'FEMALE' が含まれる場合は 1、それ以外は 0 を返す
    return 1 if 'FEMALE' in gender else 0

In [46]:
def OwnCar_to_dummy(ownCar):
    # 'あり' が含まれる場合は 1、それ以外は 0 を返す
    return 1 if 'あり' in ownCar else 0

In [47]:
def has_children(number_of_children):
    # NumberOfChildren が 1 以上なら 1 を返し、それ以外は 0 を返す
    return 1 if number_of_children >= 1 else 0

In [48]:
def function_apply(df):
    df['TypeofContact'] = df['TypeofContact'].apply(typeof_contact_to_dummy)
    df['Occupation'] = df['Occupation'].apply(occupation_to_dummy)
    df['Gender'] = df['Gender'].apply(gender_to_dummy)
    df['OwnCar'] = df['OwnCar'].apply(OwnCar_to_dummy)
    df['HasChildren'] = df['NumberOfChildren'].apply(has_children)

    return df

In [49]:
data=function_apply(data)

In [50]:
summary_df = summarize_dataframe(data)
summary_df

,列名,列の型,NaNである行の数,値の種類,NaNの率
0,id,int64,0,6978,0.000000
1,Age,float64,0,237,0.000000
2,TypeofContact,int64,0,2,0.000000
3,CityTier,int64,0,3,0.000000
4,DurationOfPitch,float64,0,285,0.000000
5,Occupation,int64,0,3,0.000000
6,Gender,int64,0,2,0.000000
7,NumberOfPersonVisiting,float64,0,4,0.000000
8,NumberOfFollowups,float64,57,7,0.008169
9,ProductPitched,object,0,5,0.000000


In [51]:
data.head()

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,MaritalStatus,OwnCar,NumberOfChildren,HasChildren
0,0,50.00000,1,2,15.0,2,0,1.0,4.0,BASIC,3.0,5.0,1,4,Executive,253905.0,1.0,未婚,0,0.0,0
1,1,56.00000,0,1,14.0,0,0,1.0,4.0,STANDARD,3.0,2.0,1,4,Senior Manager,404475.0,0.0,離婚,1,0.0,0
2,2,41.13181,1,1,10.0,2,1,1.0,3.0,BASIC,3.0,4.0,0,4,Executive,278145.0,1.0,結婚,0,0.0,0
3,3,37.00000,1,2,18.0,1,1,1.0,3.0,STANDARD,4.0,1.0,0,5,Senior Manager,326805.0,0.0,離婚,1,0.0,0
4,4,48.00000,0,3,17.0,1,1,1.0,3.0,BASIC,4.0,4.0,0,4,Executive,258435.0,1.0,独身,1,0.0,0


欠損地どうするか・・・いったん平均で

In [52]:
    def dealing_missing_values(df):
        # 欠損値を平均値で埋める
        df['Age'] = df['Age'].fillna(df['Age'].mean())
        df['NumberOfChildren'] = df['NumberOfChildren'].fillna(df['NumberOfChildren'].mean())
        df['DurationOfPitch'] = df['DurationOfPitch'].fillna(df['DurationOfPitch'].mean())
        df['NumberOfFollowups'] = df['NumberOfFollowups'].fillna(df['NumberOfFollowups'].mean())
        df['NumberOfTrips'] = df['NumberOfTrips'].fillna(df['NumberOfTrips'].mean())
        df['MonthlyIncome'] = df['MonthlyIncome'].fillna(df['MonthlyIncome'].mean())
        return df

In [53]:
  data=dealing_missing_values(data)

In [54]:
# 欠損値がある行を表示する関数
def display_missing_values(data, column_name):
    missing_data = data[data[column_name].isna()]
    print(f"欠損値がある行の数: {len(missing_data)}")
    print(f"{column_name} に欠損値があるサンプル:")
    print(missing_data)

# データの読み込みや準備（例）
# data = pd.read_csv('your_data.csv')  # ここに実際のデータ読み込みコードを記述

# 欠損値がある行を表示
display_missing_values(data, 'TypeofContact')

欠損値がある行の数: 0
TypeofContact に欠損値があるサンプル:
Empty DataFrame
Columns: [id, Age, TypeofContact, CityTier, DurationOfPitch, Occupation, Gender, NumberOfPersonVisiting, NumberOfFollowups, ProductPitched, PreferredPropertyStar, NumberOfTrips, Passport, PitchSatisfactionScore, Designation, MonthlyIncome, ProdTaken, MaritalStatus, OwnCar, NumberOfChildren, HasChildren]
Index: []


In [55]:
summary_df = summarize_dataframe(data)
summary_df

,列名,列の型,NaNである行の数,値の種類,NaNの率
0,id,int64,0,6978,0.0
1,Age,float64,0,237,0.0
2,TypeofContact,int64,0,2,0.0
3,CityTier,int64,0,3,0.0
4,DurationOfPitch,float64,0,285,0.0
5,Occupation,int64,0,3,0.0
6,Gender,int64,0,2,0.0
7,NumberOfPersonVisiting,float64,0,4,0.0
8,NumberOfFollowups,float64,0,7,0.0
9,ProductPitched,object,0,5,0.0


いったん、Agebin,Incomebin,FamilySizeはけしとくか

In [56]:
# 'Agebin', 'Incomebin', 'FamilySize' 列を削除
#data = data.drop(columns=['Agebin', 'Incomebin', 'FamilySize'])

In [57]:
summary_df = summarize_dataframe(data)
summary_df

,列名,列の型,NaNである行の数,値の種類,NaNの率
0,id,int64,0,6978,0.0
1,Age,float64,0,237,0.0
2,TypeofContact,int64,0,2,0.0
3,CityTier,int64,0,3,0.0
4,DurationOfPitch,float64,0,285,0.0
5,Occupation,int64,0,3,0.0
6,Gender,int64,0,2,0.0
7,NumberOfPersonVisiting,float64,0,4,0.0
8,NumberOfFollowups,float64,0,7,0.0
9,ProductPitched,object,0,5,0.0


In [58]:
#df_train = data[:len(train)]
#df_test = data[len(train):]
#df_test.to_csv("/content/drive/MyDrive/Signate/2024summer/cdata4/df_test_allnum.csv", index=False)
#df_train.to_csv("/content/drive/MyDrive/Signate/2024summer/cdata4/df_train_allnum.csv", index=False)

##  dammy

In [59]:
def create_dummy_col(feature, data):
    # OneHotEncoder の初期化
    ohe = OneHotEncoder(sparse_output=False, categories='auto', drop='if_binary')

    # OneHotEncoderをデータ全体に対して適用
    ohe.fit(data[[feature]])

    # ダミー変数の列名の作成
    columns = [f'{feature}_{v}' for v in ohe.categories_[0]]

    # 生成されたダミー変数をデータフレームに変換
    dummy_vals = pd.DataFrame(ohe.transform(data[[feature]]), columns=columns, index=data.index)

    # 元のデータフレームから対象の列を削除し、ダミー変数と結合
    result_data = pd.concat([data.drop([feature], axis=1), dummy_vals], axis=1)

    return result_data

In [60]:
dummy_col = ['Occupation', 'ProductPitched','Designation','MaritalStatus']

In [61]:
def apply_dummy_encoding(data, columns):
    for col in columns:
        data = create_dummy_col(col, data)
    return data

In [62]:
data = apply_dummy_encoding(data, dummy_col)

In [63]:
summary_df = summarize_dataframe(data)
summary_df

,列名,列の型,NaNである行の数,値の種類,NaNの率
0,id,int64,0,6978,0.0
1,Age,float64,0,237,0.0
2,TypeofContact,int64,0,2,0.0
3,CityTier,int64,0,3,0.0
4,DurationOfPitch,float64,0,285,0.0
5,Gender,int64,0,2,0.0
6,NumberOfPersonVisiting,float64,0,4,0.0
7,NumberOfFollowups,float64,0,7,0.0
8,PreferredPropertyStar,float64,0,3,0.0
9,NumberOfTrips,float64,0,9,0.0


標準化もするか・・

In [68]:
#std_feature = ['Age', 'DurationOfPitch', 'NumberOfPersonVisiting', 'NumberOfFollowups', 'NumberOfTrips', 'MonthlyIncome','NumberOfChildren']

In [64]:
from sklearn.preprocessing import StandardScaler

#std_sc = StandardScaler()
#ata[std_feature] = std_sc.fit_transform(data[std_feature])

In [65]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
id,6978.0,3488.500000,2014.519422,0.0,1744.25,3488.5,5232.75,6977.0
Age,6978.0,38.829420,9.715898,18.0,31.00,38.0,46.00,61.0
TypeofContact,6978.0,0.636859,0.480940,0.0,0.00,1.0,1.00,1.0
CityTier,6978.0,1.689882,0.671595,1.0,1.00,2.0,2.00,3.0
DurationOfPitch,6978.0,14.546503,6.943368,4.0,9.00,14.0,17.00,36.0
Gender,6978.0,0.395529,0.488999,0.0,0.00,0.0,1.00,1.0
NumberOfPersonVisiting,6978.0,2.401118,0.734134,1.0,2.00,2.0,3.00,4.0
NumberOfFollowups,6978.0,3.673602,0.902250,1.0,3.00,4.0,4.00,6.0
PreferredPropertyStar,6978.0,3.436515,0.599600,3.0,3.00,3.0,4.00,5.0
NumberOfTrips,6978.0,3.207261,1.727120,1.0,2.00,3.0,5.00,8.0


In [66]:
summary_df = summarize_dataframe(data)
summary_df

,列名,列の型,NaNである行の数,値の種類,NaNの率
0,id,int64,0,6978,0.0
1,Age,float64,0,237,0.0
2,TypeofContact,int64,0,2,0.0
3,CityTier,int64,0,3,0.0
4,DurationOfPitch,float64,0,285,0.0
5,Gender,int64,0,2,0.0
6,NumberOfPersonVisiting,float64,0,4,0.0
7,NumberOfFollowups,float64,0,7,0.0
8,PreferredPropertyStar,float64,0,3,0.0
9,NumberOfTrips,float64,0,9,0.0


In [ ]:
#Occupation_Designation	 わすれてた
#data = data.drop(columns=['Occupation_Designation','TravelersRatio'])

In [67]:
df_train = data[:len(train)]
df_test = data[len(train):]
#df_test.to_csv("/content/drive/MyDrive/Signate/2024summer/fdata2/df_test_allnum6.csv", index=False)
#df_train.to_csv("/content/drive/MyDrive/Signate/2024summer/fdata2/df_train_allnum6.csv", index=False)

# 新しい特徴量を作ってみる

In [68]:
data = pd.concat([df_train, df_test], sort=False)
data.head()

,id,Age,TypeofContact,CityTier,DurationOfPitch,Gender,NumberOfPersonVisiting,NumberOfFollowups,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,MonthlyIncome,ProdTaken,OwnCar,NumberOfChildren,HasChildren,Occupation_0,Occupation_1,Occupation_2,ProductPitched_BASIC,ProductPitched_DELUXE,ProductPitched_KING,ProductPitched_STANDARD,ProductPitched_SUPERDELUXE,Designation_AVP,Designation_Executive,Designation_Manager,Designation_Senior Manager,Designation_VP,MaritalStatus_未婚,MaritalStatus_独身,MaritalStatus_結婚,MaritalStatus_離婚
0,0,50.00000,1,2,15.0,0,1.0,4.0,3.0,5.0,1,4,253905.0,1.0,0,0.0,0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,56.00000,0,1,14.0,0,1.0,4.0,3.0,2.0,1,4,404475.0,0.0,1,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,2,41.13181,1,1,10.0,1,1.0,3.0,3.0,4.0,0,4,278145.0,1.0,0,0.0,0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3,37.00000,1,2,18.0,1,1.0,3.0,4.0,1.0,0,5,326805.0,0.0,1,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,4,48.00000,0,3,17.0,1,1.0,3.0,4.0,4.0,0,4,258435.0,1.0,1,0.0,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [69]:
summary_df = summarize_dataframe(data)
summary_df

,列名,列の型,NaNである行の数,値の種類,NaNの率
0,id,int64,0,6978,0.0
1,Age,float64,0,237,0.0
2,TypeofContact,int64,0,2,0.0
3,CityTier,int64,0,3,0.0
4,DurationOfPitch,float64,0,285,0.0
5,Gender,int64,0,2,0.0
6,NumberOfPersonVisiting,float64,0,4,0.0
7,NumberOfFollowups,float64,0,7,0.0
8,PreferredPropertyStar,float64,0,3,0.0
9,NumberOfTrips,float64,0,9,0.0


家族の人数

In [70]:
def add_family_size_feature(df):
    """
    家族数を新しい特徴量としてデータフレームに追加します。

    Parameters:
    - df (pd.DataFrame): データフレーム
        - 'MaritalStatus': 結婚状態 ('Married' または 'Single' など)
        - 'NumberOfChildren': 子供の数（整数）

    Returns:
    - pd.DataFrame: 家族数が追加されたデータフレーム
    """
    # 'MaritalStatus'が「結婚」または「未婚」の場合に+1し、'NumberOfChildren'を加算
    df['FamilySize'] = 1+df['NumberOfChildren'] + df['MaritalStatus_結婚']+df['MaritalStatus_未婚']

    return df
data=add_family_size_feature(data)

In [71]:
data['MonthlyIncome'] = (data['MonthlyIncome'] / 1000).astype(int)


In [ ]:
data.head()

,id,Age,TypeofContact,DurationOfPitch,Gender,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,Passport,MonthlyIncome,ProdTaken,OwnCar,NumberOfChildren,HasChildren,CityTier_1,CityTier_2,CityTier_3,Occupation_0,Occupation_1,Occupation_2,ProductPitched_BASIC,ProductPitched_DELUXE,ProductPitched_KING,ProductPitched_STANDARD,ProductPitched_SUPERDELUXE,PreferredPropertyStar_3.0,PreferredPropertyStar_4.0,PreferredPropertyStar_5.0,PitchSatisfactionScore_1,PitchSatisfactionScore_2,PitchSatisfactionScore_3,PitchSatisfactionScore_4,PitchSatisfactionScore_5,Designation_AVP,Designation_Executive,Designation_Manager,Designation_Senior Manager,Designation_VP,MaritalStatus_未婚,MaritalStatus_独身,MaritalStatus_結婚,MaritalStatus_離婚,FamilySize
0,0,50.00000,1,15.0,0,1.0,4.0,5.0,1,253,1.0,0,0.0,0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0
1,1,56.00000,0,14.0,0,1.0,4.0,2.0,1,404,0.0,1,0.0,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
2,2,41.13181,1,10.0,1,1.0,3.0,4.0,0,278,1.0,0,0.0,0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0
3,3,37.00000,1,18.0,1,1.0,3.0,1.0,0,326,0.0,1,0.0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
4,4,48.00000,0,17.0,1,1.0,3.0,4.0,0,258,1.0,1,0.0,0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


MonthlyIncome / Age

In [72]:
data['IncomePerAge'] = data['MonthlyIncome'] / data['Age']

In [73]:
data['IncomePerFamilySize'] = data['MonthlyIncome'] / data['FamilySize']

In [74]:
data['IncomePerTrips'] = data['MonthlyIncome'] / data['NumberOfTrips']

In [75]:
data['PersonTripsProduct'] = data['NumberOfPersonVisiting'] * data['NumberOfTrips']

In [76]:
data['IncomePerChild'] = data['MonthlyIncome'] / data['NumberOfChildren']

In [77]:
data['LogMonthlyIncome'] = np.log1p(data['MonthlyIncome'])

In [78]:
data['IncomePerAge'] = data['MonthlyIncome'] / data['Age']

In [79]:
data['PitchFollowupInteraction'] = data['DurationOfPitch'] * data['NumberOfFollowups']

In [80]:
data['TripsPerAge'] = data['NumberOfTrips'] / data['Age']

In [81]:
data['IncomeStarInteraction'] = data['PreferredPropertyStar'] * data['MonthlyIncome']

In [82]:
data['FollowupEfficiency'] = data['PitchSatisfactionScore'] * data['NumberOfFollowups']

In [83]:
data['IncomePerPerson'] = data['MonthlyIncome'] / (data['NumberOfPersonVisiting'] + 1)  # 顧客自身を含める

In [84]:
data['StarPerCityTier'] = data['PreferredPropertyStar'] / data['CityTier']

In [86]:
data['IncomePerStar'] = data['MonthlyIncome'] / data['PreferredPropertyStar']


In [87]:
data['LuxuryAffinity'] = data['PreferredPropertyStar'] * data['IncomePerStar']

In [89]:
data['FollowupPerTrip'] = data['NumberOfFollowups'] / data['NumberOfTrips']

In [90]:
summary_df = summarize_dataframe(data)
summary_df

,列名,列の型,NaNである行の数,値の種類,NaNの率
0,id,int64,0,6978,0.0
1,Age,float64,0,237,0.0
2,TypeofContact,int64,0,2,0.0
3,CityTier,int64,0,3,0.0
4,DurationOfPitch,float64,0,285,0.0
5,Gender,int64,0,2,0.0
6,NumberOfPersonVisiting,float64,0,4,0.0
7,NumberOfFollowups,float64,0,7,0.0
8,PreferredPropertyStar,float64,0,3,0.0
9,NumberOfTrips,float64,0,9,0.0


In [91]:
df_train = data[:len(train)]
df_test = data[len(train):]

In [92]:
df_test.to_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_test6.csv", index=False)
df_train.to_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_train6.csv", index=False)